In [48]:
# Inspired by https://www.sciencedirect.com/science/article/pii/S0377221723000735

In [49]:
import gurobipy as gp

# Create a new model
m = gp.Model()

In [50]:
# Initialize problem parameters
scraps = 3
boxes = 6

set_scraps = range(0, scraps)
set_boxes = range(0, boxes + 1)

# Box distance matrix
d = [[0, 1, 2, 3, 4, 5, 6],
    [1, 0, 1, 2, 3, 4, 5],
    [2, 1, 0, 1, 2, 3, 4],
    [3, 2, 1, 0, 1, 2, 3],
    [4, 3, 2, 1, 0, 1, 2],
    [5, 4, 3, 2, 1, 0, 1],
    [6, 5, 4, 3, 2, 1, 0]]

recipes = [[0, 1], [1, 2], [2, 0]]
set_recipes = range(0, len(recipes))

In [51]:
# Create variables

# Scrap s is assigned to box b
x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

# Box b is in solution path for recipe r
y = {}
for box in set_boxes:
    for recipe in set_recipes:
        y[box, recipe] = m.addVar(vtype='B', name=f"y[{box},{recipe}]")

# Edge b1 -> b2 is in solution path for recipe r
z = {}
for b1 in set_boxes:
    for b2 in set_boxes:
        for recipe in set_recipes:
            z[b1, b2, recipe] = m.addVar(vtype='B', name=f"z[{b1},{b2},{recipe}]")

# Box b1 precedes b2 somewhere in solution path for recipe r
p = {}
for b1 in set_boxes:
    for b2 in set_boxes:
        for recipe in set_recipes:
            p[b1, b2, recipe] = m.addVar(vtype='B', name=f"p[{b1},{b2},{recipe}]")

In [52]:
# Add constraints

# Each scrap is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes[1:]) >= 1)

# Each box is assigned at most one scrap
for b in set_boxes[1:]:
    m.addConstr(sum(x[s, b] for s in set_scraps) == 1)

# Dummy box is empty
m.addConstr(sum(x[s, 0] for s in set_scraps) == 0)

# The number of boxes in a solution for recipe r is equal to the number of scraps required by the recipe
for r in set_recipes:
    m.addConstr(sum(y[b, r] for b in set_boxes) == len(recipes[r]))

# There is exactly one box b in the solution path for recipe r that corresponds to each scrap s
for r in set_recipes:
    for s in recipes[r]:
        m.addConstr(sum(x[s, b] * y[b, r] for b in set_boxes) == 1)

# There is one incoming edge to each box b in the solution path for recipe r
for r in set_recipes:
    for b2 in set_boxes:
        m.addConstr(sum(z[b1, b2, r] for b1 in set_boxes) == y[b2, r])

# There is one outgoing edge from each box b in the solution path for recipe r
for r in set_recipes:
    for b1 in set_boxes:
        m.addConstr(sum(z[b1, b2, r] for b2 in set_boxes) == y[b1, r])

# Subtour elimination constraints
for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            m.addConstr(p[b1, b2, r] >= z[b1, b2, r])

for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            m.addConstr(p[b1, b2, r] + p[b2, b1, r] == y[b1, r] * y[b2, r])

for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            for b3 in set_boxes[1:]:
                m.addConstr(p[b1, b2, r] + p[b2, b3, r] + p[b3, b1, r] <= 2)
        

In [53]:
# Set objective function

# Test
# objective = gp.quicksum(x[1, b] for b in set_boxes)

objective = gp.quicksum(d[b1][b2] * z[b1, b2, r] for b1 in set_boxes for b2 in set_boxes for r in set_recipes)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [54]:
# Solve it!
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 811 rows, 336 columns and 2520 nonzeros
Model fingerprint: 0x972a86a6
Model has 114 quadratic constraints
Variable types: 0 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 307 rows and 75 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [55]:
# Show solution
print(f"Optimal objective value: {m.objVal}")

for v in m.getVars():
    if v.X > 0:
        print(f"{v.VarName} = {v.X}")

AttributeError: Unable to retrieve attribute 'objVal'